In [10]:
import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from copy import deepcopy
import copy
from GameDayFunctions.fangraphs_projection_2020 import Projection
#from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer')

In [208]:
class Draft:

    def __init__(self, projections_object, 
                 draft_position = 2,
                 number_teams = 12,
                 roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5},
                 batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS'],
                 pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] ):

        self.number_teams = number_teams
        self.number_rounds = sum(roster_spots.values())
        self.draft_position = draft_position - 1 # e.g., 1st pick is 0!   
        self.player_projections = projections_object
        self.remaining_ranked_players = projections_object.all_rank 
        self.roto_stats_batting = pd.DataFrame(columns =  batter_stats[1:])
        self.roto_stats_pitching = pd.DataFrame(columns =  pitcher_stats[1:])
        self.teams = {}
        # Eventually make this smarter, e.g.;
        # self.roster_spots{"fielders":{'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1},
        #                   "pitchers":{'SP':2,'RP':2,'P':3}
        #                    "bench":{'BN':5}}
        self.fielders = ['C','1B','2B','3B','SS','OF','UTIL']
        self.pitchers = ['SP', 'RP', 'P']        
        for i in np.arange(number_teams):
            roto_stats = {}
            roto_stats['batting_stats'] = pd.DataFrame(columns =  batter_stats)
            roto_stats['pitching_stats'] = pd.DataFrame(columns =  pitcher_stats)
            roto_stats['roster_spots'] = roster_spots.copy()
            roto_stats['roster'] = {}
            self.teams[i] = roto_stats
    
    # Find Resulting Standings
    def tabulate_roto(self, teams):
        roto_stats_batting = self.roto_stats_batting.copy()
        roto_stats_pitching = self.roto_stats_pitching.copy()
        for iteam in np.arange(self.number_teams):
            raw_team_batting = teams[iteam]['batting_stats']
            raw_team_pitching = teams[iteam]['pitching_stats']
            roto_team_batting = raw_team_batting.mean()
            roto_team_pitching = raw_team_pitching.mean()
            if 'AVG' in raw_team_batting:
                roto_team_batting['AVG'] = (raw_team_batting['AVG']*raw_team_batting['AB']).sum()/raw_team_batting['AB'].sum()
            if 'OPS' in raw_team_batting:
                roto_team_batting['OPS'] = (raw_team_batting['OPS']*raw_team_batting['AB']).sum()/raw_team_batting['AB'].sum() 
            if 'ERA' in raw_team_pitching:
                roto_team_batting['ERA'] = (raw_team_pitching['ERA']*raw_team_pitching['IP']).sum()/raw_team_pitching['IP'].sum() 
            if 'WHIP' in raw_team_pitching:
                roto_team_batting['WHIP'] = (raw_team_pitching['WHIP']*raw_team_pitching['IP']).sum()/raw_team_pitching['IP'].sum()
            batting_stat_names = self.roto_stats_batting.columns.values.tolist()
            pitching_stat_names = self.roto_stats_pitching.columns.values.tolist()
            roto_stats_batting = roto_stats_batting.append(roto_team_batting[batting_stat_names],ignore_index = True)
            roto_stats_pitching = roto_stats_pitching.append(roto_team_pitching[pitching_stat_names],ignore_index = True)
            
        # Combine pitching and hitting into single DataFrame.  Need to add column headings?
        roto_team_stats = pd.concat([roto_stats_batting, roto_stats_pitching], axis=1, sort=False)
        
        # Find Rank in ascending and descending order (suboptimal?) 
        roto_standings_avgs = pd.concat([roto_stats_batting.rank(ascending=False), roto_stats_pitching.rank(ascending=False).rename(columns={"BB": "BBP"})], axis=1, sort=False)
        roto_standings_cnts = pd.concat([roto_stats_batting.rank(), roto_stats_pitching.rank().rename(columns={"BB": "BBP"})], axis=1, sort=False)
        
        # Reverse rank of stats in which lower values are better. 
        avg_stats = ['CS','BBP','ERA','WHIP','BSV']
        for avg_stat in avg_stats:
            if avg_stat in roto_standings_avgs:
                roto_standings_cnts[avg_stat] = roto_standings_avgs[avg_stat]
        roto_standings = roto_standings_cnts.sum(axis=1).sort_values(ascending=False)
        roto_placement = roto_standings.index.get_loc(self.draft_position) + 1 # standings starting from 1 (not 0)
        pdb.set_trace()
        return roto_stats_batting, roto_stats_pitching, roto_standings, roto_placement
    
    def draft_into_teams(self, single_team, drafted_player, position, show = False):
                
        if show == True:
            print('Picked '+drafted_player.iloc[0].PLAYER+' for '+position)
            
        # Different Stats Entries for Pitchers and Batters
        if drafted_player.EligiblePosition.str.contains('P').bool() == True:
            idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
            statline = self.player_projections.pitchers_stats[idx_player][single_team['pitching_stats'].keys()]
            single_team['pitching_stats'] = single_team['pitching_stats'].append(statline[0:1])

        else:
            idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
            statline = self.player_projections.hitters_stats[idx_player][single_team['batting_stats'].keys()]
            single_team['batting_stats'] = single_team['batting_stats'].append(statline[0:1])        

        # Eliminate one roster_spot
        single_team['roster_spots'][position] -= 1

        # Add Player to Roster
        if position in single_team['roster']:
            single_team['roster'][position] = [single_team['roster'][position], drafted_player.PLAYER.values[0:1]]
        else:
            single_team['roster'][position] = drafted_player.PLAYER.values[0:1]
            
        return single_team
    
    def get_optimal_position(self, positions_in, roster_spots):
        # Return in the order of most valuable position to fill
        
        # Split players eligible for more that one position, e.g., 1B/OF
        single_positions = positions_in.split('/')
        
        # Check Pitchers
        P = False
        if ('RP' in single_positions):
            P = True
            if (roster_spots['RP'] > 0):
                return  'RP' 
        elif ('SP' in single_positions):
            P = True
            if (roster_spots['SP'] > 0):
                return  'SP'
        if P == True:
            if (roster_spots['P'] > 0):
                return  'P'
            elif (roster_spots['BN'] > 0):
                return 'BN'
            else:
                return 0
            
        # Check Hitters
        Util = False
        if ('C' in single_positions):
            Util = True
            if (roster_spots['C'] > 0):
                return  'C'
        elif ('2B' in single_positions): 
            Util = True
            if (roster_spots['2B'] > 0):
                return  '2B'
        elif ('SS' in single_positions):
            Util = True
            if (roster_spots['SS'] > 0):
                return  'SS'
        elif ('OF' in single_positions):
            Util = True
            if (roster_spots['OF'] > 0):
                return  'OF'
        elif ('3B' in single_positions):
            Util = True
            if (roster_spots['3B'] > 0):
                return  '3B'
        elif ('1B' in single_positions):
            Util = True 
            if (roster_spots['1B'] > 0):
                return  '1B'
        elif ('Util' in single_positions):
            Util = True 
            if (roster_spots['UTIL'] > 0):
                return  'UTIL'                
        if Util == True:
            if (roster_spots['UTIL'] > 0):
                return  'UTIL'
            elif (roster_spots['BN'] > 0):
                return 'BN'
            else:
                return 0
        else:
            return 0
        
    # Do the entire draft one round at a time
    def draft_all(self):
        for iround in np.arange(self.number_rounds):
            self.teams, self.remaining_ranked_players = self.draft_round(iround, self.teams, self.remaining_ranked_players)
            self.round_number += 1
        self.roto_stats_batting, self.roto_stats_pitching, self.roto_standings, self.roto_placement = self.tabulate_roto(self.teams)
      
    # Draft each round one team at a time.  When reaching "draft_position", stop and to pseudo_drafts to figure out best choice. 
    def draft_round(self, round_key, teams, df):
        
        # Reverse draft order every other round
        draft_order = np.arange(self.number_teams)
        if round_key % 2 == 1:
            draft_order = draft_order[::-1]
        
        # Makde deep copies so that search for best position does not write to master
        teams_copy = copy.deepcopy(teams) 
        df_copy = copy.deepcopy(df) 
        
        # Draft each round one team at a time
        for iteam in draft_order:
            
            # When team is draft_position, search for best pick.
            if iteam == self.draft_position:
                best_pick, best_position = self.find_best_pick(iteam, teams_copy, df_copy, round_key) 
                teams_copy, df_copy = self.draft_next_best(iteam, teams_copy, df_copy, force_pick = best_pick, force_position = best_position)
            else: 
                teams_copy, df_copy = self.draft_next_best(iteam, teams, df_copy)
            #teams_copy, df_copy = self.draft_next_best(iteam, teams_copy, df_copy)
            
        return teams_copy, df_copy
        
    def draft_remaining(self, teams_copy, df_copy, draft_round):

        # Draft all remaining players
        for iround in range(draft_round,self.number_rounds):
            
            draft_order = np.arange(self.number_teams)
            
            # Reverse draft order every other round
            if iround % 2 == 1:
                draft_order = draft_order[::-1]
            
            # For the very first round begin at beginning self.draft_position
            if iround == draft_round:
                if iround % 2 == 1:
                    starting_position = self.number_teams - self.draft_position;
                else:
                    starting_position = self.draft_position + 1;
                draft_order = draft_order[starting_position:] 
            
            # Finish the draft by picking the next best player in an open position 
            for iteam in draft_order:
                teams_copy, df_copy = self.draft_next_best(iteam, teams_copy, df_copy)

        return teams_copy, df_copy
        
    def find_best_pick(self, team_key, teams_copy, df_copy, round_key, silent == True):
        # find_best_pick returns iloc, the index (of df) of the optimal pick, and the position being filled

        # Determine which roster_spots are still unfilled
        unfilled_positions = [k for (k,v) in teams_copy[team_key]['roster_spots'].items() if v > 0]    
        idx_eligible = []
        pos_eligible = []
        
        # Find index of best player at each remaining position
        for iunfilled in unfilled_positions:
            if iunfilled == 'UTIL':
                idx_position = [i for i, val in enumerate(df_copy.EligiblePosition.str.contains('|'.join(self.fielders))) if val]
            elif iunfilled == 'P':
                idx_position = [i for i, val in enumerate(df_copy.EligiblePosition.str.contains('|'.join(self.pitchers))) if val]
            elif iunfilled == 'BN':
                idx_position = [i for i, val in enumerate(df_copy.EligiblePosition.str.contains('|'.join(self.fielders+self.pitchers))) if val]                
            else:
                idx_position = [i for i, val in enumerate(df_copy.EligiblePosition.str.contains(iunfilled)) if val] 
            filled = False
            jdx = 0
            while filled == False:
                if idx_position[jdx] in idx_eligible:
                    jdx+=1
                else:
                    idx_eligible.append(idx_position[jdx])
                    pos_eligible.append(iunfilled)
                    filled = True

        # Get rid of doubles (1B and OF is particularly prone)
        idx_eligible, idx_unique = np.unique(idx_eligible, return_index = True)
        pos_eligible = [pos_eligible[i] for i in idx_unique]
        if silent == False:
            print('Picking from:')
            print(df_copy.iloc[idx_eligible])
        
        #################################
        # START OF LOOP TO FIND BEST PLAYER
        player_based_drafted_outcomes = {}
        player_based_drafted_teams = {}

        # Loop over eligible players, then finish the draft
        for iposition, icounter in zip(idx_eligible, range(len(idx_eligible))):
            
            # make a copy of teams to finish drafting
            teams_loop = copy.deepcopy(teams_copy) 
            df_loop = copy.deepcopy(df_copy) #df_copy.copy()
            
            # Get iplayer before dropping
            iplayer = df_loop.iloc[iposition].PLAYER

            # Draft looping through idx_eligible
            df_loop,drafted_player=df_loop.drop(df_loop.iloc[iposition:iposition+1].index),df_loop.iloc[iposition:iposition+1]
            position = pos_eligible[icounter]
    
            teams_loop[team_key] = self.draft_into_teams(teams_loop[team_key], drafted_player, position, show = False)

            # LOOP OVER WHOLE REST OF THE DRAFT HERE...
            teams_loop, df_loop = self.draft_remaining(teams_loop, df_loop, round_key)
            
            # Calculate the best pseudo-standings
            #pseudo_batting_stats, pseudo_pitching_stats, pseudo_standings, pseudo_placement = self.tabulate_roto(teams_loop)
            roto_stats = self.tabulate_roto(teams_loop)
            
            # Store the result. 
            player_based_drafted_teams[iplayer] = teams_loop[self.draft_position]['roster']
            player_based_drafted_outcomes[iplayer] = [roto_stats[3],roto_stats[2][roto_stats[3]-1]]
            if silent == False:
                print('Stored Result for Pick '+str(icounter)+' '+iplayer+' '+pos_eligible[icounter]+' whose score is '+str(player_based_drafted_outcomes[iplayer]))

        # End of Loop 
        ranked_positions = ['C','2B','SS','OF','3B','1B','SP','RP','UTIL','P','BN']
        
        # Decide on best choice and return 
        relative_ranking = [player_based_drafted_outcomes[i][0] for i in player_based_drafted_outcomes]
        relative_ranking_rank = np.argsort(relative_ranking)
        relative_scores = [player_based_drafted_outcomes[i][1] for i in player_based_drafted_outcomes]

        # If there is a tie for top relative_ranking, select by highest score, then optimal position
        n_max_ranking = sum(relative_ranking == np.min(relative_ranking))
        if n_max_ranking == 1:
            best_player = df_copy.iloc[idx_eligible[relative_ranking_rank[0]]]
            best_pick_plus_one = idx_eligible[relative_ranking_rank[0]] + 1 # Avoid best_pick = 0
            best_position = pos_eligible[relative_ranking_rank[0]]             
        else:
            # Of those tied for top rank, figure out had a highest score
            best_player_scores = [relative_scores[relative_ranking_rank[i]] for i in range(n_max_ranking)]
            best_players = [df_copy.iloc[idx_eligible[relative_ranking_rank[i]]] for i in range(n_max_ranking)]
            best_picks_plus_one = [idx_eligible[relative_ranking_rank[i]] + 1 for i in range(n_max_ranking)]
            best_player_positions = [pos_eligible[relative_ranking_rank[i]] for i in range(n_max_ranking)]
            idx_best_player_scores = np.argsort(best_player_scores)[::-1]
            
            # If still tied, take the optimal position (i.e., SS over OF)
            n_max_scores = sum(best_player_scores == np.max(best_player_scores))
            if n_max_scores == 1:
                best_player = df_copy.iloc[best_picks_plus_one[idx_best_player_scores[0]]-1:best_picks_plus_one[idx_best_player_scores[0]]-1+1]
                best_pick_plus_one = best_picks_plus_one[idx_best_player_scores[0]]
                best_position = best_player_positions[idx_best_player_scores[0]]
            else:
                best_player_positions = [pos_eligible[idx_best_player_scores[i]] for i in range(n_max_scores)] 
                for irank in range(len(ranked_positions)):
                    if any(ranked_positions[irank] in s for s in best_player_positions):
                        idx_best = best_player_positions.index(ranked_positions[irank])
                        best_player = df_copy.iloc[best_picks_plus_one[idx_best_player_scores[idx_best]]-1:best_picks_plus_one[idx_best_player_scores[idx_best]]-1+1]
                        best_pick_plus_one = best_picks_plus_one[idx_best_player_scores[idx_best]]
                        best_position = ranked_positions[irank]
                        break
                                           
        try:
            print('Best Pick is '+ best_player)
        except:
            pdb.set_trace()
                
        # Need to not fill UTIL if other opening exist...
        if (best_position == 'UTIL'):
            pdb.set_trace()
        #if (best_position == 'UTIL') & any(ranked_positions[irank] in s for s in best_player_positions):
        
        return best_pick_plus_one, best_position
        # END OF LOOP TO FIND BEST PLAYER
        #################################
    
    # Strategy is to take the best possible player, even if that means putting them in UTIL or BN (maybe BN should reconsidered...)
    def draft_next_best(self, team_key, teams, df, force_pick = False, force_position = False, silent = True):
        
        if (force_pick == False):
            pick_ok = False
            idf = 0
            while (pick_ok == False):
                df_copy = copy.deepcopy(df) #df.apply(deepcopy)
                pick_ok = True

                # Draft next in list.  Do this way (idf) so that if you don't take player, they are not removed for next picker
                df_copy,drafted_player=df_copy.drop(df_copy.iloc[idf:idf+1].index),df_copy.iloc[idf:idf+1]
                    
                # Get eligible positions
                eligible_positions = drafted_player.EligiblePosition.values[0]
                
                # Find best position opening.  Return 0 if no room
                position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
                
                # Update Rosters with drafted_player, Loop otherwise
                if position == 0:
                    unfilled_positions = [k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]           
                    if silent == False:
                        print('Not Drafting '+eligible_positions+" "+drafted_player.PLAYER+' for '+ "/".join(unfilled_positions))
                    idf += 1
                    if len(unfilled_positions) > 0:
                        pick_ok = False
                    else:
                        pick_ok = True
                else:
                    if silent == False:
                        print('Team '+ str(team_key) +' Drafting '+drafted_player.iloc[0].PLAYER+' for '+position)                    
                    df = df_copy                            
                    pick_ok = True
                    teams[team_key] = self.draft_into_teams(teams[team_key], drafted_player,position, show = False)
                    
        else:
            print('Forcing Pick')
            pick_ok = True
            pick = force_pick - 1 
            df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]
            position = force_position
      
            teams[team_key] = self.draft_into_teams(teams[team_key], drafted_player, position, show = False)
            print('Team '+ str(team_key) +' force BEST pick '+drafted_player.iloc[0].PLAYER+' for '+position)

        return teams, df

In [218]:
zipsM= Draft(player_ranking_zips, number_teams = 6, roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2})
zipsM.draft_all()

Stored Result for Pick 0 Mookie Betts OF whose score is [2, 79.0]
Stored Result for Pick 1 Mike Trout UTIL whose score is [2, 79.0]
Stored Result for Pick 2 Ronald Acuña BN whose score is [2, 77.0]
Stored Result for Pick 3 Anthony Rendon 3B whose score is [2, 75.0]
Stored Result for Pick 4 Cody Bellinger 1B whose score is [3, 76.0]
Stored Result for Pick 5 Alex Bregman SS whose score is [1, 77.0]
Stored Result for Pick 6 Justin Verlander SP whose score is [3, 78.0]
Stored Result for Pick 7 Jacob deGrom P whose score is [3, 78.5]
Stored Result for Pick 8 Whit Merrifield 2B whose score is [3, 83.0]
Stored Result for Pick 9 Roberto Osuna RP whose score is [3, 84.0]
Stored Result for Pick 10 J.T. Realmuto C whose score is [2, 83.5]
PLAYER              Best Pick is Alex Bregman
EligiblePosition           Best Pick is 3B/SS
Name: 9, dtype: object
Forcing Pick
Team 1 force BEST pick Alex Bregman for SS
Stored Result for Pick 0 Justin Verlander SP whose score is [1, 77.0]
Stored Result for Pic

In [ ]:
zipsS= Draft(player_ranking_zips, number_teams = 4, roster_spots = {'C':0,'1B':0,'2B':1, '3B':1,'SS':1,'OF':2,'UTIL':1,'SP':1,'RP':1,'P':1,'BN':1})

In [215]:
steamM= Draft(player_ranking_zips, number_teams = 6, roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':2})
steam = Draft(player_ranking_steam)

In [216]:
steam.draft_all()

Stored Result for Pick 0 Mookie Betts OF whose score is [12, 137.0]
Stored Result for Pick 1 Mike Trout UTIL whose score is [8, 165.0]
Stored Result for Pick 2 Ronald Acuña BN whose score is [12, 138.0]
Stored Result for Pick 3 Anthony Rendon 3B whose score is [12, 142.0]
Stored Result for Pick 4 Cody Bellinger 1B whose score is [10, 127.5]
Stored Result for Pick 5 Alex Bregman SS whose score is [10, 132.0]
Stored Result for Pick 6 Justin Verlander SP whose score is [8, 156.0]
Stored Result for Pick 7 Jacob deGrom P whose score is [8, 155.0]
Stored Result for Pick 8 Whit Merrifield 2B whose score is [10, 163.0]
Stored Result for Pick 9 Roberto Osuna RP whose score is [11, 133.0]
Stored Result for Pick 10 J.T. Realmuto C whose score is [2, 150.0]
PLAYER              Best Pick is J.T. Realmuto
EligiblePosition                Best Pick is C
Name: 185, dtype: object
Forcing Pick
Team 1 force BEST pick J.T. Realmuto for C
Stored Result for Pick 0 Jack Flaherty SP whose score is [2, 150.0]
S

In [202]:
zipsS.draft_all()

Stored Result for Pick 0 Mookie Betts OF whose score is [3, 51.0]
Stored Result for Pick 1 Mike Trout UTIL whose score is [1, 54.5]
Stored Result for Pick 2 Ronald Acuña BN whose score is [2, 53.0]
Stored Result for Pick 3 Anthony Rendon 3B whose score is [2, 52.0]
Stored Result for Pick 4 Alex Bregman SS whose score is [4, 40.0]
Stored Result for Pick 5 Justin Verlander SP whose score is [2, 50.0]
Stored Result for Pick 6 Jacob deGrom P whose score is [2, 48.5]
Stored Result for Pick 7 Whit Merrifield 2B whose score is [4, 41.0]
Stored Result for Pick 8 Roberto Osuna RP whose score is [2, 51.5]
PLAYER              Best Pick is Mike Trout
EligiblePosition            Best Pick is OF
Name: 3, dtype: object
Forcing Pick
Team 1 force BEST pick Mike Trout for UTIL
Stored Result for Pick 0 Cody Bellinger OF whose score is [1, 54.5]
Stored Result for Pick 1 Freddie Freeman BN whose score is [3, 50.5]
Stored Result for Pick 2 Alex Bregman 3B whose score is [3, 50.5]
Stored Result for Pick 3 Ju

In [203]:
zipsM.draft_all()

Stored Result for Pick 0 Mookie Betts OF whose score is [2, 79.0]
Stored Result for Pick 1 Mike Trout UTIL whose score is [2, 79.0]
Stored Result for Pick 2 Ronald Acuña BN whose score is [2, 77.0]
Stored Result for Pick 3 Anthony Rendon 3B whose score is [2, 75.0]
Stored Result for Pick 4 Cody Bellinger 1B whose score is [3, 76.0]
Stored Result for Pick 5 Alex Bregman SS whose score is [1, 77.0]
Stored Result for Pick 6 Justin Verlander SP whose score is [3, 78.0]
Stored Result for Pick 7 Jacob deGrom P whose score is [3, 78.5]
Stored Result for Pick 8 Whit Merrifield 2B whose score is [3, 83.0]
Stored Result for Pick 9 Roberto Osuna RP whose score is [3, 84.0]
Stored Result for Pick 10 J.T. Realmuto C whose score is [2, 83.5]
PLAYER              Best Pick is Alex Bregman
EligiblePosition           Best Pick is 3B/SS
Name: 9, dtype: object
Forcing Pick
Team 1 force BEST pick Alex Bregman for SS
Stored Result for Pick 0 Justin Verlander SP whose score is [1, 77.0]
Stored Result for Pic

In [213]:
zips.teams[1]['roster']

{'1B': array(['Cody Bellinger'], dtype=object),
 'SP': [array(['Walker Buehler'], dtype=object),
  array(['Zack Greinke'], dtype=object)],
 'SS': array(['Trea Turner'], dtype=object),
 'BN': [[[[array(['Stephen Strasburg'], dtype=object),
     array(['Trevor May'], dtype=object)],
    array(['Matt Manning'], dtype=object)],
   array(['César Hernández'], dtype=object)],
  array(['Chris Bassitt'], dtype=object)],
 '2B': array(['Jeff McNeil'], dtype=object),
 '3B': array(['Eduardo Escobar'], dtype=object),
 'RP': [array(['Yusmeiro Petit'], dtype=object),
  array(['Rafael Montero'], dtype=object)],
 'P': [[array(['Blake Snell'], dtype=object),
   array(['Tommy Kahnle'], dtype=object)],
  array(['Lucas Giolito'], dtype=object)],
 'UTIL': array(['Gleyber Torres'], dtype=object),
 'C': array(['J.T. Realmuto'], dtype=object),
 'OF': [[array(['Nick Senzel'], dtype=object),
   array(['David Fletcher'], dtype=object)],
  array(['Willie Calhoun'], dtype=object)]}

In [217]:
zips.roto_standings

1     165.5
0     165.0
5     139.0
10    136.0
2     133.5
11    126.0
8     124.0
7     122.5
4     122.0
9     118.0
3     111.0
6      97.5
dtype: float64

In [212]:
zips.roto_placement

1

In [210]:
zips = Draft(player_ranking_zips)
zips.draft_all()

Stored Result for Pick 0 Mookie Betts OF whose score is [4, 92.5]
Stored Result for Pick 1 Mike Trout UTIL whose score is [6, 115.0]
Stored Result for Pick 2 Ronald Acuña BN whose score is [11, 132.0]
Stored Result for Pick 3 Anthony Rendon 3B whose score is [11, 129.0]
Stored Result for Pick 4 Cody Bellinger 1B whose score is [3, 126.5]
Stored Result for Pick 5 Alex Bregman SS whose score is [4, 106.5]
Stored Result for Pick 6 Justin Verlander SP whose score is [7, 95.5]
Stored Result for Pick 7 Jacob deGrom P whose score is [9, 111.0]
Stored Result for Pick 8 Whit Merrifield 2B whose score is [9, 126.0]
Stored Result for Pick 9 Roberto Osuna RP whose score is [10, 186.0]
Stored Result for Pick 10 J.T. Realmuto C whose score is [7, 110.5]
PLAYER              Best Pick is Cody Bellinger
EligiblePosition             Best Pick is 1B/OF
Name: 7, dtype: object
Forcing Pick
Team 1 force BEST pick Cody Bellinger for 1B
Stored Result for Pick 0 Walker Buehler SP whose score is [3, 126.5]
Stor

In [199]:
zips.teams[4]['roster']

{'OF': [[array(['Ronald Acuña'], dtype=object),
   array(['Yordan Alvarez'], dtype=object)],
  array(['Aaron Judge'], dtype=object)],
 'SP': [array(['Max Scherzer'], dtype=object),
  array(['Corey Kluber'], dtype=object)],
 'RP': [array(['Seth Lugo'], dtype=object),
  array(['Will Smith'], dtype=object)],
 'UTIL': array(['Michael Conforto'], dtype=object),
 'P': [array(['Ian Kennedy'], dtype=object),
  array(['Craig Kimbrel'], dtype=object)]}

In [7]:
steam.draft_all()

In [8]:
steam.roto_placement

9

In [8]:
zips.roto_standings

0     171.0
9     163.0
7     156.0
1     144.5
10    143.0
4     123.0
6     117.0
11    114.5
3     114.0
8     108.0
2     103.5
5     102.5
dtype: float64

In [9]:
steam.roto_standings

0     180.0
9     162.0
10    153.0
11    140.5
1     130.0
4     125.0
7     122.0
3     122.0
2     116.5
6     115.0
5     114.0
8      80.0
dtype: float64

In [10]:
steam.teams[6]['roster']

{'1B': 'Cody Bellinger',
 'SS': 'Fernando Tatis',
 '2B': 'Ozzie Albies',
 'RP': ['Aroldis Chapman', 'Tyler Duffey'],
 'UTIL': 'Carlos Santana',
 'SP': ['Drew Pomeranz', 'Brandon Woodruff'],
 'P': [['Zack Britton', 'John Brebbia'], 'Colin Poche'],
 'BN': [[[['Yoan López', 'Keynan Middleton'], 'Mark Melancon'], 'Luis Patiño'],
  'Tyler Rogers'],
 '3B': 'Luis Arraez',
 'OF': [['Nick Senzel', 'Willie Calhoun'], 'Franmil Reyes'],
 'C': 'Carson Kelly'}

In [ ]:
    
    # Strategy is to take the best possible player, even if that means putting them in UTIL or BN (maybe BN should reconsidered...)
    def draft_next_best_old(self, team_key, teams = 'teams', df = pd.DataFrame(), force_pick = False):
                
        if bool(teams == 'teams'):
            teams = self.teams
        
        if df.empty == True:
            df = self.remaining_ranked_players

        if (force_pick == False):
            pick_ok = False
            idf = 0
            while pick_ok == False:
                df_copy = df.copy()
                pick_ok = True

                df_copy,drafted_player=df_copy.drop(df_copy.iloc[idf:idf+1].index),df_copy.iloc[idf:idf+1]
                eligible_positions = drafted_player.EligiblePosition.values[0]
                

                if eligible_positions == 'Util':
                    open_positions = [teams[team_key]['roster_spots']['UTIL']]
                else:
                    open_positions = [teams[team_key]['roster_spots'][i] for i in eligible_positions.split('/')]
                
                # If there are no SP, RP, or Fielding Positions left, try P, UTIL, or BN
                if np.sum(open_positions) == 0:
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        if teams[team_key]['roster_spots']['P'] > 0:
                            position = 'P'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False
                            
                    else:
                        if teams[team_key]['roster_spots']['UTIL'] > 0:
                            position = 'UTIL'
                            df = df_copy
                        elif teams[team_key]['roster_spots']['BN'] > 0:
                            position = 'BN'
                            df = df_copy
                        else:
                            position = 'No Roster Spots Left'
                            pick_ok = False

                else:

                    df = df_copy                            
                    position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
                    #print(drafted_player.PLAYER+' is a '+position)
                    pick_ok = True

                if pick_ok == True:
                    # Check if Position is a Pitcher or Batter
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                        teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

                    else:
                        idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                        teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])
                        #pdb.set_trace()
                    #print('Team '+ str(team_key) +' Drafting '+drafted_player.iloc[0].PLAYER+' for '+position)
                else:
                    #print('Not Drafting '+drafted_player.PLAYER+' for '+position)
                    # Try while loop again after incrementing idf
                    idf += 1

        else:
            #pdb.set_trace()
            pick = force_pick - 1 
            df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]
            position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
            
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][teams[team_key]['pitching_stats'].keys()]
                teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][teams[team_key]['batting_stats'].keys()]
                teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])        
                
            print('Team '+ str(team_key) +' FORCED to pick '+drafted_player.iloc[0].PLAYER+' for '+position)
        
        # Eliminate one roster_spot
        teams[team_key]['roster_spots'][position] -= 1

        # Add Player to Roster
        if position in self.teams[team_key]['roster']:
            teams[team_key]['roster'][position] = [teams[team_key]['roster'][position], drafted_player.PLAYER.values[0]]
        else:
            teams[team_key]['roster'][position] = drafted_player.PLAYER.values[0]
            
        return teams, df

In [ ]:

    # Strategy is to take the best possible player, even if that means putting them in UTIL or BN (maybe BN should reconsidered...)
    def draft_next_best_second(self, team_key, teams = 'teams', df = pd.DataFrame(), force_pick = False):
        
        #pdb.set_trace()
        if bool(teams == 'teams'):
            teams = self.teams.copy()
        
        if df.empty == True:
            df = self.player_projections.all_rank #self.remaining_ranked_players.copy()

        df.head()
        if (force_pick == False):
            pick_ok = False
            idf = 0
            while (pick_ok == False) & (len([k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]) > 0):
                df_copy = df.copy()
                pick_ok = True

                # Draft next in list.  Do this way (idf) so that if you don't take player, they are not removed for next picker
                df_copy,drafted_player=df_copy.drop(df_copy.iloc[idf:idf+1].index),df_copy.iloc[idf:idf+1]
                
                # Get eligible positions
                eligible_positions = drafted_player.EligiblePosition.values[0]
                
                # Find best position opening.  Return 0 if no room
                position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
                
                # Update Rosters with drafted_player, Loop otherwise
                if position == 0:
                    unfilled_positions = [k for (k,v) in self.teams[team_key]['roster_spots'].items() if v > 0]           
                    #print('Not Drafting '+eligible_positions+" "+drafted_player.PLAYER+' for '+ "/".join(unfilled_positions))
                    idf += 1
                    pick_ok = False
                else:
                    #print('Team '+ str(team_key) +' Drafting '+drafted_player.iloc[0].PLAYER+' for '+position)                    
                    df = df_copy                            
                    pick_ok = True
                    # Check if Position is a Pitcher or Batter
                    if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                        idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.pitchers_stats[idx_player][self.teams[team_key]['pitching_stats'].keys()]
                        teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

                    else:
                        idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                        statline = self.player_projections.hitters_stats[idx_player][self.teams[team_key]['batting_stats'].keys()]
                        teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])
                        #pdb.set_trace()
        else:
            pdb.set_trace()
            pick = force_pick -1 
            df,drafted_player=df.drop(df.iloc[pick:pick+1].index),df.iloc[pick:pick+1]
            eligible_positions = drafted_player.EligiblePosition.values[0]
            position = self.get_optimal_position(eligible_positions, teams[team_key]['roster_spots'])
            
            if drafted_player.EligiblePosition.str.contains('P').bool() == True:
                idx_player = self.player_projections.pitchers_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.pitchers_stats[idx_player][teams[team_key]['pitching_stats'].keys()]
                teams[team_key]['pitching_stats'] = teams[team_key]['pitching_stats'].append(statline[0:1])

            else:
                idx_player = self.player_projections.hitters_stats.Name.values == drafted_player.iloc[0].PLAYER
                statline = self.player_projections.hitters_stats[idx_player][teams[team_key]['batting_stats'].keys()]
                teams[team_key]['batting_stats'] = teams[team_key]['batting_stats'].append(statline[0:1])        
                
            print('Team '+ str(team_key) +' force BEST pick '+drafted_player.iloc[0].PLAYER+' for '+position)
        
        # Eliminate one roster_spot
        teams[team_key]['roster_spots'][position] -= 1

        # Add Player to Roster
        if position in self.teams[team_key]['roster']:
            teams[team_key]['roster'][position] = [teams[team_key]['roster'][position], drafted_player.PLAYER.values[0]]
        else:
            teams[team_key]['roster'][position] = drafted_player.PLAYER.values[0]
            
        return teams, df